# ***Installation***

In [4]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

     |████████████████████████████████| 1.6 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 946 kB 5.3 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# ***Importing libraries***

In [6]:
import autogluon as ag
import pandas as pd
import numpy as np
import os,urllib
from autogluon.tabular import  TabularPredictor

## ***Loading Dataset***

In [24]:
train = pd.read_csv("iris_train.csv")
test = pd.read_csv("iris_test.csv")

In [25]:
train.drop("Unnamed: 0",axis=1,inplace=True)
test.drop("Unnamed: 0",axis=1,inplace=True)

# ***EDA***

In [26]:
train.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [28]:
test.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [29]:
round((train.isnull().sum() * 100/ len(train)),2).sort_values(ascending=False)

species         0.0
petal_width     0.0
petal_length    0.0
sepal_width     0.0
sepal_length    0.0
dtype: float64

In [30]:
round((test.isnull().sum() * 100/ len(test)),2).sort_values(ascending=False)

species         0.0
petal_width     0.0
petal_length    0.0
sepal_width     0.0
sepal_length    0.0
dtype: float64

In [31]:
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
sepal_length,120.0,5.845000,0.868578,4.4,5.075,5.8,6.425,7.9
sepal_width,120.0,3.065000,0.427156,2.0,2.800,3.0,3.300,4.4
petal_length,120.0,3.739167,1.822100,1.0,1.500,4.4,5.100,6.9
petal_width,120.0,1.196667,0.782039,0.1,0.300,1.3,1.800,2.5
species,120.0,1.000000,0.840168,0.0,0.000,1.0,2.000,2.0


In [32]:
test.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
sepal_length,30.0,5.836667,0.653628,4.3,5.500,5.75,6.300,7.1
sepal_width,30.0,3.010000,0.463383,2.2,2.725,3.00,3.300,4.2
petal_length,30.0,3.836667,1.537459,1.1,2.300,4.25,4.900,5.9
petal_width,30.0,1.206667,0.694775,0.1,0.625,1.30,1.575,2.5
species,30.0,1.000000,0.742781,0.0,0.250,1.00,1.750,2.0


In [33]:
label_column = 'species'
train[label_column].describe()

count    120.000000
mean       1.000000
std        0.840168
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        2.000000
Name: species, dtype: float64

# ***Building Model***

In [39]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label_column, path=save_path).fit(train)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.3.1
Train Data Rows:    120
Train Data Columns: 4
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	3 unique label values:  [2, 1, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12684.8 MB
	Train Data (Original)  Memory Usage: 0.0 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:


# ***Predictions***

In [42]:
y_test = test[label_column]

test_data_nolab = test.drop(labels=[label_column],axis=1) 
print(test_data_nolab.head())


   sepal_length  sepal_width  petal_length  petal_width
0           5.9          3.0           4.2          1.5
1           6.9          3.1           5.4          2.1
2           5.1          3.3           1.7          0.5
3           6.0          3.4           4.5          1.6
4           5.5          2.5           4.0          1.3


In [45]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.9666666666666667
Evaluations on test data:
{
    "accuracy": 0.9666666666666667,
    "balanced_accuracy": 0.9761904761904763,
    "mcc": 0.9501212424878318
}


Predictions:  
 0     1
1     2
2     0
3     1
4     1
5     1
6     0
7     2
8     1
9     2
10    2
11    0
12    2
13    1
14    1
15    0
16    1
17    0
18    0
19    2
20    0
21    1
22    2
23    2
24    1
25    1
26    0
27    1
28    2
29    1
Name: species, dtype: int64


# ***Leaderboard***

In [46]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,0.966667,1.000000,0.003042,0.002747,0.244154,0.003042,0.002747,0.244154,1,True,13
1,LightGBM,0.966667,0.958333,0.004308,0.002672,0.155895,0.004308,0.002672,0.155895,1,True,5
2,CatBoost,0.966667,0.958333,0.004871,0.001223,0.186900,0.004871,0.001223,0.186900,1,True,8
3,XGBoost,0.966667,0.958333,0.020085,0.003797,0.152663,0.020085,0.003797,0.152663,1,True,11
4,NeuralNetFastAI,0.966667,1.000000,0.024868,0.037757,14.654864,0.024868,0.037757,14.654864,1,True,3
5,KNeighborsUnif,0.966667,1.000000,0.103095,0.109746,0.023452,0.103095,0.109746,0.023452,1,True,1
6,KNeighborsDist,0.966667,1.000000,0.103575,0.106587,0.009065,0.103575,0.106587,0.009065,1,True,2
7,WeightedEnsemble_L2,0.966667,1.000000,0.106391,0.118530,0.454659,0.003296,0.008784,0.431207,2,True,14
8,ExtraTreesEntr,0.966667,0.958333,0.113986,0.103681,0.871104,0.113986,0.103681,0.871104,1,True,10
9,RandomForestEntr,0.966667,0.958333,0.114913,0.102882,0.877080,0.114913,0.102882,0.877080,1,True,7


# ***Summary***

In [47]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         LightGBMLarge   1.000000       0.002747   0.244154                0.002747           0.244154            1       True         13
1       NeuralNetFastAI   1.000000       0.037757  14.654864                0.037757          14.654864            1       True          3
2        KNeighborsDist   1.000000       0.106587   0.009065                0.106587           0.009065            1       True          2
3        KNeighborsUnif   1.000000       0.109746   0.023452                0.109746           0.023452            1       True          1
4   WeightedEnsemble_L2   1.000000       0.118530   0.454659                0.008784           0.431207            2       True         14
5              CatBoost   0.958333       0.001223   0.186900                0.001223           0.186900 